In [108]:
import json
import gzip

import psycopg2

### Take home test
for fetch

In [386]:
# RESET SQL HERE

conn = psycopg2.connect(dbname='postgres',
                        host='localhost',
                        user='postgres',
                        password='123456')

cur = conn.cursor()

In [378]:
# First unzip the raw data

url_brands = 'data/brands.json.gz'
url_receipts = 'data/receipts.json.gz'
url_users = 'data/users.json.gz'

with gzip.open(url_receipts, 'r') as file:
    bytes_data = file.read()
    data_str = bytes_data.decode('utf-8')

# Separate different rows of data, list of strings
data = data_str.split('\n')

# fist data row
json.loads(data[1])

{'_id': {'$oid': '5ff1e1bb0a720f052300056b'},
 'bonusPointsEarned': 150,
 'bonusPointsEarnedReason': 'Receipt number 5 completed, bonus point schedule DEFAULT (5cefdcacf3693e0b50e83a36)',
 'createDate': {'$date': 1609687483000},
 'dateScanned': {'$date': 1609687483000},
 'finishedDate': {'$date': 1609687483000},
 'modifyDate': {'$date': 1609687488000},
 'pointsAwardedDate': {'$date': 1609687483000},
 'pointsEarned': '150.0',
 'purchaseDate': {'$date': 1609601083000},
 'purchasedItemCount': 2,
 'rewardsReceiptItemList': [{'barcode': '4011',
   'description': 'ITEM NOT FOUND',
   'finalPrice': '1',
   'itemPrice': '1',
   'partnerItemId': '1',
   'quantityPurchased': 1},
  {'barcode': '028400642255',
   'description': 'DORITOS TORTILLA CHIP SPICY SWEET CHILI REDUCED FAT BAG 1 OZ',
   'finalPrice': '10.00',
   'itemPrice': '10.00',
   'needsFetchReview': True,
   'needsFetchReviewReason': 'USER_FLAGGED',
   'partnerItemId': '2',
   'pointsNotAwardedReason': 'Action not allowed for user an

In [379]:
len(data)

1120

In [380]:
# Useful recursive function to flatten any given JSON row
# Copied from https://www.geeksforgeeks.org/flattening-json-objects-in-python/

def flatten_json(y):
    out = {}
  
    def flatten(x, name =''):
          
        # If the Nested key-value 
        # pair is of dict type
        if type(x) is dict:
              
            for a in x:
                flatten(x[a], name + a + '_')
                  
        # If the Nested key-value
        # pair is of list type
        elif type(x) is list:
              
            i = 0
              
            for a in x:                
                flatten(a, name + str(i) + '_')
                i += 1
        else:
            out[name[:-1]] = x
  
    flatten(y)
    return out

In [357]:
flatten_json(json.loads(data[1]))

{'_id_$oid': '5ff1e1bb0a720f052300056b',
 'bonusPointsEarned': 150,
 'bonusPointsEarnedReason': 'Receipt number 5 completed, bonus point schedule DEFAULT (5cefdcacf3693e0b50e83a36)',
 'createDate_$date': 1609687483000,
 'dateScanned_$date': 1609687483000,
 'finishedDate_$date': 1609687483000,
 'modifyDate_$date': 1609687488000,
 'pointsAwardedDate_$date': 1609687483000,
 'pointsEarned': '150.0',
 'purchaseDate_$date': 1609601083000,
 'purchasedItemCount': 2,
 'rewardsReceiptItemList_0_barcode': '4011',
 'rewardsReceiptItemList_0_description': 'ITEM NOT FOUND',
 'rewardsReceiptItemList_0_finalPrice': '1',
 'rewardsReceiptItemList_0_itemPrice': '1',
 'rewardsReceiptItemList_0_partnerItemId': '1',
 'rewardsReceiptItemList_0_quantityPurchased': 1,
 'rewardsReceiptItemList_1_barcode': '028400642255',
 'rewardsReceiptItemList_1_description': 'DORITOS TORTILLA CHIP SPICY SWEET CHILI REDUCED FAT BAG 1 OZ',
 'rewardsReceiptItemList_1_finalPrice': '10.00',
 'rewardsReceiptItemList_1_itemPrice': 

In [295]:
# Making the schema for the brands table
query = '''
        CREATE TABLE brands (
            _id VARCHAR(24), 
            barcode BIGINT, 
            brandCode VARCHAR(64),
            category VARCHAR(64),
            categoryCode VARCHAR(64),
            cpg VARCHAR(24),
            topBrand BOOL,
            name VARCHAR(64)
        );
        '''
cur.execute(query)

In [296]:
def row_to_sql(num):
    # Returns a list of values (as strings) at a given row 
    # so it can fit into SQL
    
    row = flatten_json(json.loads(data[num]))

    keys = ['_id_$oid', 'barcode', 'brandCode', 'category', 'categoryCode',
            'cpg_$ref', 'topBrand', 'name']

    vals = ['NULL'] * len(keys)

    # All 'NULL' unless data is available
    for i, k in enumerate(keys):
        if k in row.keys():
            a = row[k]
            if isinstance(a, str):
                a = a.replace("'", '')
            vals[i] = a
    
    return vals

In [297]:
row_to_sql(8)

['5ab15636e4b0be0a89bb0b07',
 '511111504412',
 'CAMPBELLS HOME STYLE',
 'Canned Goods & Soups',
 'NULL',
 'Cogs',
 False,
 'Campbells Home Style']

In [298]:
flatten_json(json.loads(data[1167]))

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [299]:
for i in range(len(data)-1):
    vals = row_to_sql(i)

    query = '''
            INSERT INTO brands
            VALUES (
            '{0}', '{1}', '{2}', '{3}', '{4}', '{5}', {6}, '{7}'
            );
            '''.format(*vals)
    
    # Replacing 'NULL' varchar with NULL type
    query = query.replace("'NULL'", "NULL")

    cur.execute(query)

0
1
2
3
4


In [278]:
query = '''
        SELECT category, COUNT(category) FROM brands GROUP BY category;
        '''

cur.execute(query)

In [279]:
cur.fetchall()

[('Dairy', 33),
 ('Magazines', 44),
 ('Snacks', 75),
 ('Deli', 6),
 ('Canned Goods & Soups', 12),
 ('NULL', 155),
 ('Condiments & Sauces', 27),
 ('Grocery', 39),
 ('Beauty & Personal Care', 6),
 ('Candy & Sweets', 71),
 ('Health & Wellness', 44),
 ('Bread & Bakery', 5),
 ('Baby', 18),
 ('Baking', 369),
 ('Cleaning & Home Improvement', 6),
 ('Beverages', 63),
 ('Outdoor', 1),
 ('Beer Wine Spirits', 90),
 ('Dairy & Refrigerated', 5),
 ('Breakfast & Cereal', 40),
 ('Household', 5),
 ('Frozen', 24),
 ('Personal Care', 20),
 ('Beauty', 9)]

In [256]:
data[1167]

''

In [300]:
query = '''
        SELECT * FROM brands LIMIT 4;
        '''

cur.execute(query)
cur.fetchall()

[('601ac115be37ce2ead437551',
  511111019862,
  None,
  'Baking',
  'BAKING',
  'Cogs',
  False,
  'test brand @1612366101024'),
 ('601c5460be37ce2ead43755f',
  511111519928,
  'STARBUCKS',
  'Beverages',
  'BEVERAGES',
  'Cogs',
  False,
  'Starbucks'),
 ('601ac142be37ce2ead43755d',
  511111819905,
  'TEST BRANDCODE @1612366146176',
  'Baking',
  'BAKING',
  'Cogs',
  False,
  'test brand @1612366146176'),
 ('601ac142be37ce2ead43755a',
  511111519874,
  'TEST BRANDCODE @1612366146051',
  'Baking',
  'BAKING',
  'Cogs',
  False,
  'test brand @1612366146051')]

In [381]:
def populate_sql(url, table):
    '''docstring'''
    
    # Start with Python's connection to PostgreSQL database
    # Note: must have a postgres docker container running locally

    conn = psycopg2.connect(dbname='postgres',
                            host='localhost',
                            user='postgres',
                            password='123456')
    
    curr = conn.cursor()
    
    # Defining some variables that change depending on the table
    if table == 'brands':
        keys = ['_id_$oid', 'barcode', 'brandCode', 'category', 
                'categoryCode', 'cpg_$ref', 'topBrand', 'name']
        schema = '''
            CREATE TABLE brands (
                _id VARCHAR(24), 
                barcode BIGINT, 
                brandCode VARCHAR(64),
                category VARCHAR(64),
                categoryCode VARCHAR(64),
                cpg VARCHAR(24),
                topBrand BOOL,
                name VARCHAR(64)
            );
            '''
        empty_query = '''
            INSERT INTO brands
            VALUES (
            '{0}', '{1}', '{2}', '{3}', '{4}', '{5}', {6}, '{7}'
            );
            '''
    elif table == 'users':
        keys = ['_id_$oid', 'state', 'createdDate_$date', 'lastLogin_$date',
                'role', 'active']
        schema = '''
            CREATE TABLE users (
                _id VARCHAR(24),
                state VARCHAR(2),
                createdDate VARCHAR(13),
                lastLogin VARCHAR(13),
                role VARCHAR(12),
                active BOOL
            );
            '''
        empty_query = '''
            INSERT INTO users
            VALUES (
            '{0}', '{1}', '{2}', '{3}', '{4}', {5}
            );
            '''
    elif table == 'receipts':
        keys = ['_id_$oid', 'bonusPointsEarned', 'bonusPointsEarnedReason',
                'createDate_$date', 'dateScanned_$date', 'finishedDate_$date',
                'modifyDate_$date', 'pointsAwardedDate_$date', 'pointsEarned',
                'purchaseDate_$date', 'purchasedItemCount',
                'rewardsReceiptStatus', 'totalSpent', 'userId']
        schema = '''
            CREATE TABLE receipts (
                _id VARCHAR(24),
                bonusPointsEarned INT,
                bonusPointsEarnedReason VARCHAR(128),
                createDate VARCHAR(13),
                dateScanned VARCHAR(13),
                finishedDate VARCHAR(13),
                modifyDate VARCHAR(13),
                pointsAwardedDate VARCHAR(13),
                pointsEarned REAL,
                purchaseDate VARCHAR(13),
                purchasedItemCount INT,
                rewardsReceiptStatus VARCHAR(12),
                totalSpent REAL,
                userId VARCHAR(24)
            );
            '''
        empty_query = '''
            INSERT INTO receipts
            VALUES (
            '{0}', '{1}', '{2}', '{3}', '{4}', '{5}', {6}, '{7}',
            '{8}', '{9}', '{10}', '{11}', '{12}', '{13}'
            );
            '''
    else:
        print("Couldn't tell what data you wanted")
        return None
    
    cur.execute(schema)
    
    # Now data. First unzip raw data
    with gzip.open(url, 'r') as file:
        bytes_data = file.read()
        data_str = bytes_data.decode('utf-8')

    # Separate different rows of data, list of strings
    data = data_str.split('\n')

    # Loops through each "row" and inserts into SQL database
    for i in range(len(data)-1):
        # Get data from row
        vals = row_to_sql(data, i, keys)
        
        # Populate empty query with row data
        query = empty_query.format(*vals)

        # Replacing 'NULL' varchar with NULL type
        query = query.replace("'NULL'", "NULL")

        cur.execute(query)

In [382]:
# Useful recursive function to flatten any given JSON row
# Copied from https://www.geeksforgeeks.org/flattening-json-objects-in-python/

def flatten_json(y):
    out = {}
  
    def flatten(x, name =''):
          
        # If the Nested key-value 
        # pair is of dict type
        if type(x) is dict:
              
            for a in x:
                flatten(x[a], name + a + '_')
                  
        # If the Nested key-value
        # pair is of list type
        elif type(x) is list:
              
            i = 0
              
            for a in x:                
                flatten(a, name + str(i) + '_')
                i += 1
        else:
            out[name[:-1]] = x
  
    flatten(y)
    return out

In [383]:
def row_to_sql(data, num, keys):
    # Returns a list of values (as strings) at a given row 
    # so it can fit into SQL
    
    row = flatten_json(json.loads(data[num]))

    vals = ['NULL'] * len(keys)

    # All 'NULL' unless data is available
    for i, k in enumerate(keys):
        if k in row.keys():
            a = row[k]
            if isinstance(a, str):
                a = a.replace("'", '')
            vals[i] = a
    
    return vals

In [387]:
populate_sql(url='data/users.json.gz', table='users')
populate_sql(url='data/brands.json.gz', table='brands')
#populate_sql(url='data/receipts.json.gz', table='receipts')

In [389]:
query = '''
        SELECT * FROM receipts LIMIT 4;
        '''

cur.execute(query)
cur.fetchall()

[('5ff1e1eb0a720f0523000575',
  500,
  'Receipt number 2 completed, bonus point schedule DEFAULT (5cefdcacf3693e0b50e83a36)',
  '1609687531000',
  '1609687531000',
  '1609687531000',
  '1609687536000',
  '1609687531000',
  500.0,
  '1609632000000',
  5,
  'FINISHED',
  26.0,
  '5ff1e1eacfcf6c399c274ae6'),
 ('5ff1e1bb0a720f052300056b',
  150,
  'Receipt number 5 completed, bonus point schedule DEFAULT (5cefdcacf3693e0b50e83a36)',
  '1609687483000',
  '1609687483000',
  '1609687483000',
  '1609687488000',
  '1609687483000',
  150.0,
  '1609601083000',
  2,
  'FINISHED',
  11.0,
  '5ff1e194b6a9d73a3a9f1052'),
 ('5ff1e1f10a720f052300057a',
  5,
  'All-receipts receipt bonus',
  '1609687537000',
  '1609687537000',
  None,
  '1609687542000',
  None,
  5.0,
  '1609632000000',
  1,
  'REJECTED',
  10.0,
  '5ff1e1f1cfcf6c399c274b0b'),
 ('5ff1e1ee0a7214ada100056f',
  5,
  'All-receipts receipt bonus',
  '1609687534000',
  '1609687534000',
  '1609687534000',
  '1609687539000',
  '1609687534000',


In [388]:
conn.commit()

#### in terminal commands

If you don't have the docker container
> docker run -d -p 5432:5432 --name my-postgres -e POSTGRES_PASSWORD=123456 postgres

Otherwise open existing docker container
> docker run my-postgres

Run python script (only needs to be run once)
> python fetch.py

within terminal (local)
> docker exec -it my-postgres bash

within terminal (container)
will ask for password (123456)
> psql -h localhost -p 5432 -U postgres -W

see what databases are present (should include fetch)
> \l

connect to Fetch database
> \c postgres

list tables
> \dt

execute queries